In [1]:
import numpy as np
from ldpc import bposd_decoder
from ldpc.codes import rep_code 
from bposd.hgp import hgp
import tqdm
import time
from qecsim import paulitools as pt 
from qecsim.models.generic import DepolarizingErrorModel
from qecsim.models.toric import ToricCode
import galois
from typing import List, Tuple
import nbimporter
import BP_OSD_Simulation_V2 as my_bposd

In [28]:
dim = 6
my_code = ToricCode(dim,dim)
my_error_model = DepolarizingErrorModel()
GF = galois.GF(2)
num_qubits = 2 * dim ** 2
error_probability = 0.08
Z_stabs = (my_code.stabilizers[:dim ** 2])[:, 2 * dim ** 2:]
X_stabs = (my_code.stabilizers[dim ** 2:])[:, :2 * dim ** 2]

In [30]:
import numpy as np
from ldpc import bposd_decoder

bpd_X=bposd_decoder(
    Z_stabs,#the parity check matrix
    error_rate=error_probability,
    channel_probs=[None], #assign error_rate to each qubit. This will override "error_rate" input variable
    max_iter=num_qubits, #the maximum number of iterations for BP)
    bp_method="ms",
    ms_scaling_factor=0, #min sum scaling factor. If set to zero the variable scaling factor method is used
    osd_method="osd_cs", #the OSD method. Choose from:  1) "osd_e", "osd_cs", "osd0"
    osd_order=7 #the osd search depth
    )

bpd_Z=bposd_decoder(
    X_stabs,#the parity check matrix
    error_rate=error_probability,
    channel_probs=[None], #assign error_rate to each qubit. This will override "error_rate" input variable
    max_iter=num_qubits, #the maximum number of iterations for BP)
    bp_method="ms",
    ms_scaling_factor=0, #min sum scaling factor. If set to zero the variable scaling factor method is used
    osd_method="osd_cs", #the OSD method. Choose from:  1) "osd_e", "osd_cs", "osd0"
    osd_order=7 #the osd search depth
    )

In [18]:
rng = np.random.default_rng()
# Error: random error based on error probability
error = my_error_model.generate(my_code, error_probability, rng)
while (sum(error) == 0):
    error = my_error_model.generate(my_code, error_probability, rng)

Z_error = error[2 * dim**2:]
X_error = error[:2 * dim**2]
Z_syndrome = np.mod(X_stabs @ Z_error,2)
X_syndrome = np.mod(Z_stabs @ X_error, 2)
bpd_X.decode(syndromeX)
bpd_Z.decode(syndromeZ)

print("X Error")
print(x_error)
print("Z Error")
print(z_error)
print("BP+OSD Decoding X")
print(bpd_X.osdw_decoding)
print("BP+OSD Decoding Z")
print(bpd_Z.osdw_decoding)
#Decoding is successful if the residual error commutes with the logical operators
recovery_x = (bpd_X.osdw_decoding) %2
recovery_z = (bpd_Z.osdw_decoding) %2
residual_error_x = (recovery_x + x_error) %2
residual_error_z = (recovery_z + z_error) %2

print("Commutation with Stabs")
print(np.sum((surface_code.hz@residual_error_x) % 2))
print(np.sum((surface_code.hx@residual_error_z) % 2))

X Error
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
Z Error
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
BP+OSD Decoding X
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 